In [17]:
import os

In [18]:
import cv2
import numpy as np

In [19]:
def read_data(csv_path):
    with open(csv_path,"r") as file:
        lines = file.read().splitlines()
        return [line.split(",") for line in lines]


In [20]:
train_data = read_data("train.csv")
test_data = read_data("test.csv")

In [21]:
def read_and_resize(path, size):
    bgr = cv2.imread(path, cv2 .IMREAD_COLOR)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return cv2.resize(rgb, size)

In [22]:
def load_data(data):
    image_list = []
    label_list = []
    for path, name in data:
        image = read_and_resize(path, (32, 32))
        name = int(name)
        image_list.append(image)
        label_list.append(name)
    return np.stack(image_list,axis=0), np.array(label_list)

In [23]:
x_train, y_train = load_data(train_data)
x_test, y_test = load_data(test_data)

In [24]:
def label2onehot(label):
    return np.eye(2)[label]

In [25]:
y_train_onehot = label2onehot(y_train)
y_test_onehot = label2onehot(y_test)

In [26]:
x_train = x_train /255
x_test = x_test /255

In [27]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(3160, 32, 32, 3) (3160,)
(789, 32, 32, 3) (789,)


# Model

In [28]:
from tensorflow import keras

In [29]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256))
model.add(keras.layers.Activation(keras.activations.linear))
model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation(keras.activations.linear))
model.add(keras.layers.Dense(2))
model.add(keras.layers.Activation(keras.activations.softmax))

loss_func = keras.losses.categorical_crossentropy
metrics = [
    keras.metrics.categorical_accuracy
]
optimizer = keras.optimizers.Adam(lr=0.0001)

model.compile(loss=loss_func, optimizer=optimizer, metrics=metrics)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
activation_9 (Activation)    (None, 16, 16, 32)       

## Train

In [30]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.ModelCheckpoint("best_model.h5",monitor='val_loss',
                                   mode = 'min',
                                   save_best_only = True)
]


In [31]:
his = model.fit(x_train, y_train_onehot,
                validation_data=(x_test, y_test_onehot),
                epochs = 15,
                batch_size=32,
                verbose=1,
                callbacks=callbacks
        )

Epoch 1/15
99/99 [==============================] - 11s 107ms/step - loss: 0.3397 - categorical_accuracy: 0.8487 - val_loss: 0.1486 - val_categorical_accuracy: 0.9430
Epoch 2/15
99/99 [==============================] - 9s 92ms/step - loss: 0.1241 - categorical_accuracy: 0.9566 - val_loss: 0.1122 - val_categorical_accuracy: 0.9544
Epoch 3/15
99/99 [==============================] - 9s 93ms/step - loss: 0.0951 - categorical_accuracy: 0.9690 - val_loss: 0.1032 - val_categorical_accuracy: 0.9556
Epoch 4/15
99/99 [==============================] - 9s 94ms/step - loss: 0.0844 - categorical_accuracy: 0.9744 - val_loss: 0.1159 - val_categorical_accuracy: 0.9556
Epoch 5/15
99/99 [==============================] - 10s 97ms/step - loss: 0.0757 - categorical_accuracy: 0.9744 - val_loss: 0.1037 - val_categorical_accuracy: 0.9544
Epoch 6/15
99/99 [==============================] - 10s 103ms/step - loss: 0.0767 - categorical_accuracy: 0.9725 - val_loss: 0.1016 - val_categorical_accuracy: 0.9632
Epoch

In [32]:
val_loss, val_acc = model.evaluate(x_test,y_test_onehot)

25/25 [==============================] - 1s 20ms/step - loss: 0.0804 - categorical_accuracy: 0.9696
